In [2]:
from yaml import safe_load, safe_dump
from geopy.distance import geodesic
from maro.simulator.scenarios.oncall_routing import Coordinate
import os
import random

import pandas as pd

def geo_distance_meter(source_coordinate: Coordinate, target_coordinate: Coordinate) -> float:
    return geodesic(source_coordinate, target_coordinate).m

station_coord = Coordinate(lat=32.72329226, lng=-117.0718922)

dir_name = "/data/songlei/maro/maro/simulator/scenarios/oncall_routing/topologies/example_clean/"
route_df = pd.read_csv(f"/data/songlei/maro/maro/simulator/scenarios/oncall_routing/topologies/example/routes.csv")
route_df.head()

route_df_list = []
for route_name in route_df["ROUTENBR"].unique():
    _df = route_df[route_df['ROUTENBR'] == route_name].sort_values(by='STOPORDER').reset_index()
    new_lat, new_lng = [], []
    pre_loc = None
    for i, row in _df.iterrows():
        if i == 0:
            new_lat.append(row['LAT'])
            new_lng.append(row['LNG'])
        else:
            cur_coord = Coordinate(lat=row['LAT'], lng=row['LNG'])
            dist = geo_distance_meter(pre_loc, cur_coord)
            if dist >= 2e3:
                new_lat.append(pre_loc.lat+random.random() / 100000.0)
                new_lng.append(pre_loc.lng+random.random() / 100000.0)
            else:
                new_lat.append(row['LAT'])
                new_lng.append(row['LNG'])
        pre_loc = Coordinate(lat=new_lat[-1], lng=new_lng[-1])
    _df.loc[:, 'LAT'] = new_lat
    _df.loc[:, "LNG"] = new_lng
    route_df_list.append(_df)
df = pd.concat(route_df_list)
df.to_csv(f"{dir_name}/routes.csv", index=False)
        
with open(os.path.join("/data/songlei/maro/maro/simulator/scenarios/oncall_routing/topologies/example", "oncall_info.yml")) as fp:
    oncall_info = safe_load(fp)
    oncall_coords = oncall_info["coordinates"]
    oncall_open_times = oncall_info["open_times"]
    oncall_time_windows = oncall_info["time_windows"]
    oncall_order_num = oncall_info["oncall_numbers"]


for i in range(len(oncall_coords[0])):
    cur_coord = Coordinate(lat=oncall_coords[0][i][0], lng=oncall_coords[0][i][1])
    try:
        if geo_distance_meter(station_coord, cur_coord) >= 5e4:
            oncall_coords[0][i][0] = station_coord.lat + random.random() / 100000.0
            oncall_coords[0][i][1] = station_coord.lng + random.random() / 100000.0
    except:
        print(station_coord, cur_coord)
        
for i in range(len(oncall_open_times[0])):
    if oncall_open_times[0][i] >= 1300 or oncall_open_times[0][i] <= 800:
        oncall_open_times[0][i] = random.randint(800, 1300)
        
for i in range(len(oncall_time_windows[0])):
    if oncall_time_windows[0][i] <= 120:
        oncall_time_windows[0][i] = random.randint(120, 360)
        
for i in range(len(oncall_order_num)):
    if oncall_order_num[i] >= 70 or oncall_order_num[i] <= 40:
        oncall_order_num[i] = random.randint(50, 70)


with open(os.path.join(dir_name, "oncall_info.yml"), 'w') as fp:
    oncall_info["coordinates"] = oncall_coords
    oncall_info["open_times"] = oncall_open_times
    oncall_info["time_windows"] = oncall_time_windows
    oncall_info["oncall_numbers"] = oncall_order_num
    safe_dump(oncall_info, fp)


In [4]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

from typing import List, Optional, Tuple, Dict

from maro.simulator import Env
from maro.simulator.scenarios.oncall_routing import Coordinate
from maro.simulator.scenarios.oncall_routing.common import Action, PostponeAction, OncallRoutingPayload, AllocateAction
from maro.simulator.scenarios.oncall_routing.order import OrderStatus
from maro.utils import set_seeds
import pandas as pd
from greedy2 import _get_actions
# from greedy_with_time_window import _get_actions
# from greedy import _get_actions

from examples.oncall_routing.utils import refresh_segment_index

set_seeds(2048)

oncall_id_list = []

def order_status2_str(order_status):
    if order_status == OrderStatus.NOT_READY:
        return "NotReady"
    elif order_status == OrderStatus.READY_IN_ADVANCE:
        return "ReadyInAdvance"
    elif order_status == OrderStatus.IN_PROCESS:
        return "InProcess"
    elif order_status == OrderStatus.IN_PROCESS_BUT_DELAYED:
        return "Delayed"
    elif order_status == OrderStatus.COMPLETED:
        return "Complete"
    elif order_status == OrderStatus.TERMINATED:
        return "Terminated"
    else:
        return "Dummy"

def collect_info(running_env: Env, event: OncallRoutingPayload):
    result_list = []
    tick = running_env.tick
    route_meta_info_dict = running_env.business_engine._get_route_meta_info_dict(tick)
    carriers_in_stop: List[bool] = (running_env.snapshot_list["carriers"][tick::"in_stop"] == 1).tolist()
    carrier_position_dict: Dict[str, Tuple[bool, int, int, Coordinate]] = {
        route_name: [
            carriers_in_stop[meta["carrier_idx"]],
            meta["estimated_duration_to_the_next_stop"],
            meta["estimated_next_departure_tick"],
            meta["current_staying_stop_coordinate"]
        ]
        for route_name, meta in route_meta_info_dict.items()
    }
    finished_orders = running_env.business_engine.get_finished_orders_dict()
    for route_name in finished_orders.keys():
        meta = route_meta_info_dict[route_name]
        for order, finished_tick in finished_orders[route_name]:
            result = [tick, route_name, 'COMPLETED', 
                      order.coord[0], 
                      order.coord[1], 
                      finished_tick,
                      order.open_time,
                      order.close_time,
                      order.is_delivery,
                      order_status2_str(order._status),
                      meta['carrier_idx'],
                      (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][0]),
                      (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][1]),
                      order.id
                     ]
            result_list.append(result)
    if event:
        oncall_orders = event.oncall_orders
        route_plan_dict = event.route_plan_dict
        for route_name, meta in route_meta_info_dict.items():
            planned_orders = route_plan_dict[route_name]
            for i, planned_order in enumerate(planned_orders):
                result = [tick, route_name, 'PLANNED', 
                        planned_order.coord[0], 
                        planned_order.coord[1], 
                        tick,
                        planned_order.open_time,
                        planned_order.close_time,
                        planned_order.is_delivery,
                        order_status2_str(planned_order.get_status(tick, env.business_engine._config.order_transition)),
                        meta['carrier_idx'],
                        (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][0]),
                        (-1 if carrier_position_dict[route_name][3] is None else carrier_position_dict[route_name][3][1]),
                        planned_order.id
                        ]
                result_list.append(result)
        for oncall_order in oncall_orders:
            oncall_id_list.append(oncall_order.id)
            result = [tick, "Not Assigned", 'ONCALL', 
                    oncall_order.coord[0], 
                    oncall_order.coord[1], 
                    tick,
                    oncall_order.open_time,
                    oncall_order.close_time,
                    oncall_order.is_delivery,
                    order_status2_str(oncall_order.get_status(tick, env.business_engine._config.order_transition)),
                    -1,
                    -1,
                    -1,
                    oncall_order.id
                    ]
            result_list.append(result)
    df_orders = pd.DataFrame(data=result_list, 
                             columns=['tick', 'route_name', 'type',
                                      'order_lat', 'order_lng', 'finish_tick', 
                                      'open_time', 'close_time',
                                      'is_delivery', 'order_status',
                                      'carrier_id', 'carrier_lat', 'carrier_lng', 'order_id'])
    # df_orders.to_csv('hist_orders.csv', index=False)
    return df_orders


# Greedy: assign each on-call order to the closest stop on existing route.
if __name__ == "__main__":
    env = Env(
        scenario="oncall_routing", topology="example_clean", start_tick=480, durations=960,
    )
    print(env._business_engine._max_tick)
    print(env._business_engine._config.order_transition)
    env.reset(keep_seed=True)
    metrics, decision_event, is_done = env.step(None)
    df_list = []
    while not is_done:
        assert isinstance(decision_event, OncallRoutingPayload)
        df_orders = collect_info(env, decision_event)
        df_list.append(df_orders)
        print(f"Processing {len(decision_event.oncall_orders)} oncall orders at tick {env.tick}.")
        # metrics, decision_event, is_done = env.step(_get_actions(env, decision_event))
        # for oncall_order in decision_event.oncall_orders:
        metrics, decision_event, is_done = env.step(_get_actions(env, decision_event))
    df_orders = collect_info(env, None)
    df_list.append(df_orders)
    df = pd.concat(df_list)
    df.loc[:, 'type'] = df.apply(lambda row: ("ONCALL" if row['order_id'] in oncall_id_list else "PLANNED"), axis=1)
    df.to_csv("hist_routing_orders.csv", index=False)
    print(env.tick)  
    print(env.metrics) 
    

Config path: /data/songlei/maro/maro/simulator/scenarios/oncall_routing/topologies/example_clean
Loading oncall orders.
oncall 60
Loading plans.
Loading routes data from /data/songlei/maro/maro/simulator/scenarios/oncall_routing/topologies/example_clean/routes.csv.
Loading finished. Loaded data of 100 routes.
1440
{'buffer_before_open_time': 30, 'buffer_after_open_time': 30, 'last_tick_for_order_processing': 1440, 'processing_time': 1, 'processing_proportion_to_quantity': False}
oncall 60
Processing 36 oncall orders at tick 501.
Processing 1 oncall orders at tick 509.
Processing 1 oncall orders at tick 520.
Processing 1 oncall orders at tick 525.
Processing 2 oncall orders at tick 528.
Processing 1 oncall orders at tick 531.
Processing 2 oncall orders at tick 537.
Processing 1 oncall orders at tick 546.
Processing 1 oncall orders at tick 552.
Processing 2 oncall orders at tick 556.
Processing 2 oncall orders at tick 558.
Processing 1 oncall orders at tick 567.
Processing 2 oncall order

In [3]:
df_cnt = df[((df['tick']==1439) & (df['type'] == "ONCALL"))].groupby('route_name').count().reset_index()
df_cnt.sort_values(by='order_id', inplace=True, ascending=False)
df_cnt.head(20)

KeyError: 'tick'

In [3]:
df_cnt = df[((df['tick']==1439))].groupby('route_name').count().reset_index()
df_cnt.sort_values(by='order_id', inplace=True, ascending=False)
df_cnt.head(20)

,route_name,tick,type,order_lat,order_lng,finish_tick,open_time,close_time,is_delivery,order_status,carrier_id,carrier_lat,carrier_lng,order_id
59,667,130,130,130,130,130,130,130,129,130,130,130,130,130
78,918,126,126,126,126,126,126,126,125,126,126,126,126,126
77,917,124,124,124,124,124,124,124,123,124,124,124,124,124
84,929,122,122,122,122,122,122,122,121,122,122,122,122,122
86,937,120,120,120,120,120,120,120,119,120,120,120,120,120
60,669,118,118,118,118,118,118,118,117,118,118,118,118,118
34,548,118,118,118,118,118,118,118,117,118,118,118,118,118
50,638,117,117,117,117,117,117,117,116,117,117,117,117,117
29,528,117,117,117,117,117,117,117,116,117,117,117,117,117
79,919,115,115,115,115,115,115,115,114,115,115,115,115,115
